In [18]:
import pandas as pd
import pycountry
from sqlalchemy import create_engine
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()


## National Footprint Accounts 2018 from kaggle.com

In [34]:
NFA_df=pd.read_csv("NFA_2018.csv")
print(NFA_df.shape)
print(NFA_df.columns)
NFA_df.head()
NFA_df=NFA_df.rename(columns={'ISO alpha-3 code':'alpha_3'})

(87020, 15)
Index(['country', 'ISO alpha-3 code', 'UN_region', 'UN_subregion', 'year',
       'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total', 'Percapita GDP (2010 USD)',
       'population'],
      dtype='object')


In [37]:
# create country/ountry_code table and clean it
countries_df=NFA_df[["alpha_3","country"]].drop_duplicates(keep="first")
print(countries_df.shape)
# find missing values in  column
print(len(countries_df["alpha_3"].unique()) == len(countries_df["country"].unique()))

drop_list=countries_df[countries_df["alpha_3"].isna()]["country"]
print(drop_list)
countries_df=countries_df[~(countries_df["country"].isin(drop_list))]

(196, 2)
False
23020      Ethiopia PDR
69780    Sudan (former)
85700             World
Name: country, dtype: object


In [4]:
NFA_2_df=NFA_df[~(NFA_df["country"].isin(drop_list))]

In [5]:
regions_df=NFA_2_df[['country','UN_region','UN_subregion']].drop_duplicates(keep="first").reset_index(drop=True)

In [6]:
population_df=NFA_2_df[["country","year","population"]].drop_duplicates(keep="first").reset_index(drop=True)

In [7]:
records_list_df=NFA_df["record"].drop_duplicates(keep="first")

In [8]:
e_footprint_df=NFA_2_df[["country","year",'record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon']].loc[NFA_df["record"].isin(["BiocapTotGHA"," EFConsTotGHA","EFExportsTotGHA","EFImportsTotGHA","EFProdTotGHA"])]

In [43]:
df_names=[countries_df,regions_df,population_df,e_footprint_df]
table_names=["countries","regions","population","e_footprint"]

## Use pandas to load csv converted DataFrame into database

In [44]:
rds_connection_string = "root:mohammad@localhost/national_footprint"
engine = create_engine(f'mysql://{rds_connection_string}')

In [48]:
for i in range(len(df_names)):
    df_names[i].to_sql(name =table_names[i],if_exists = 'replace', con = engine, index = False)

### Confirm data has been added by querying the customer_location table
pd.read_sql_query('select * from regions', con=engine).head()

,country,UN_region,UN_subregion
0,Armenia,Asia,Western Asia
1,Afghanistan,Asia,Southern Asia
2,Albania,Europe,Southern Europe
3,Algeria,Africa,Northern Africa
4,Angola,Africa,Middle Africa


## countries' land area from wikipedia page

In [58]:
url="https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area"
country_area_df=pd.read_html(url)[0]
print(country_area_df.head())
# clean the country_area_df 
columns=list(country_area_df.loc[0])
#print(columns)
columns=['Rank', 'sovereign state/dependency', 'Total_in_km2(mi2)', 'Land_in_km2(mi2)', 'Water_in_km2(mi2)', '%water', 'Notes']
country_area_df.columns=columns
country_area_df=country_area_df.iloc[2:]
country_area_df=country_area_df.drop(columns="Rank").reset_index(drop=True).sort_values("sovereign state/dependency")
print(country_area_df.shape)
country_area_df.head()

      0                           1                   2                  3  \
0  Rank  Sovereign state/dependency  Total in km2 (mi2)  Land in km2 (mi2)   
1   NaN                       World       (196,940,000)       (57,510,000)   
2   NaN                      Russia         (6,601,670)        (6,323,482)   
3   NaN                  Antarctica         (5,400,000)        (5,400,000)   
4   NaN                      Canada         (3,855,100)        (3,511,023)   

                    4        5  \
0  Water in km2 (mi2)  % water   
1       (139,434,000)     70.8   
2           (278,200)     4.21   
3                 (0)        0   
4           (344,080)     8.93   

                                                   6  
0                                              Notes  
1                                                NaN  
2  Largest country in the world (10.995% of the w...  
3  13,720,000 km2 (5,300,000 sq mi) (98%) of land...  
4  Largest English and French-speaking country an..

,sovereign state/dependency,Total_in_km2(mi2),Land_in_km2(mi2),Water_in_km2(mi2),%water,Notes
174,Abkhazia,"(3,340)",NaN,NaN,NaN,[Note 49]
42,Afghanistan,"(251,830)","(251,830)",(0),0,NaN
224,Akrotiri and Dhekelia (United Kingdom),(98.0),not determined,not determined,not determined,There is a salt lake and some wetlands of unkn...
146,Albania,"(11,100)","(10,578)",(520),4.7,NaN
10,Algeria,"(919,595)","(919,595)",(0),0,Largest Arabic-speaking and Berber-speaking co...


In [ ]:
trial=pd.merge(iso_country,countries_df,how="outer",on="country").sort_values("country").reset_index(drop=True)
trial4=trial3[trial3.isnull().any(axis=1)]
print(trial4)

SyntaxError: EOL while scanning string literal (<ipython-input-53-70cbaa790ce2>, line 1)

## World CO2 emissions from consumption of energy from theguardian.com

In [ ]:
co2 = pd.read_excel('World CO2 emissions from consumption of energy.xlsx',sheet_name= 'Total Carbon Dioxide Emissions')

In [ ]:
co2.columns = co2.iloc[0,]

In [ ]:
co2.reset_index(inplace= True)

In [ ]:
co2 = co2.reindex(co2.index.drop(0))

In [ ]:
co2.rename(columns = {'level_0':'rank_2009','level_1':'rank_2008','Rank, 2006':'rank_2006',
                     'ISO country code': 'iso_code'},inplace = True)
co2.columns.name = None

In [ ]:
clean = co2.loc[~co2['iso_code'].isnull(),]

In [ ]:
# Table!! Create the ISO Code and Country List
iso_country = clean[['iso_code','Country']].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'Country':'country','index':'id'})
iso_country.head()

# remove duplicated isocode and year combinations
iso_country = iso_country.loc[~iso_country.duplicated(subset=['iso_code','country'])]

# UM map to both 'Wake Island' and U.S. Pacific Islands. After research, update UM for U.S. Pacific Islands to UM1
#iso_country[iso_country['country'] == 'U.S. Pacific Islands']['iso_code'] = 'UM1'
print(iso_country.head())
#iso_country[iso_country["iso_code"] == "CN"]["country"]="chinaa"


In [59]:
trial3=pd.merge(iso_country,countries_df,how="outer",on="country").sort_values("country").reset_index(drop=True)
trial4=trial3[trial3.isnull().any(axis=1)]
print(trial4)

NameError: name 'iso_country' is not defined

In [ ]:
##### rank_list = ['rank_2009','rank_2008','rank_2006','% change,  2008 to 2009','% change 2000 to 2009','Change in place, 2008 to 2009']
clean_rank_change = clean[[*rank_list,'iso_code']].set_index('iso_code')
clean_rank_change.head()

In [ ]:
#remove the percentage changes
clean_rank_new = clean_rank_change[['rank_2009','rank_2008','rank_2006']].rename(columns = lambda x : x.replace('rank_',''))

clean_rank_new = clean_rank_new.stack().reset_index().rename(columns = {'level_1':'year',0:'rank'})


clean_rank_new = clean_rank_new[~clean_rank_new.duplicated(subset = ['iso_code','year'])]
clean_rank_new.head()

In [ ]:
##Table!! Create the Country, Year, and CO2 Emission table
clean_co2 = clean.drop(rank_list,axis = 1)
clean_co2 = clean_co2.set_index('iso_code').drop('Country',axis = 1)
clean_co2 = clean_co2.stack().reset_index().rename(columns = {'level_1':'year',0:'co2_emission'})

# remove duplicated isocode and year combinations
clean_co2 = clean_co2.loc[~clean_co2.duplicated(subset=['iso_code','year'])]

In [21]:
len(list(pycountry.countries))

249